In [1]:
import pandas as pd

In [2]:
data = pd.read_excel(r'data.xlsx')

In [3]:
data

,num,name,p_np,smiles
0,1,Propanolol,p,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,p,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,p,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,p,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,p,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...
...,...,...,...,...
2048,2049,licostinel,p,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2049,2050,ademetionine(adenosyl-methionine),p,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2050,2051,mesocarb,p,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2051,2052,tofisoline,p,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...


In [4]:
activity = []
for row in data['p_np']:
    if row == 'p':
        activity.append(1)
    else:
        activity.append(0)

In [5]:
activity[0:5]

[1, 1, 1, 1, 1]

In [6]:
len(activity)

2053

In [7]:
min(activity)

0

In [8]:
data['activity'] = activity

In [9]:
to_be_printed = data[['smiles', 'activity']]

In [10]:
to_be_printed.describe()

,activity
count,2053.000000
mean,0.764735
std,0.424268
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [11]:
to_be_printed = to_be_printed.rename(columns={'smiles': 'SMILES', 'activity': 'target'})

In [12]:
to_be_printed.head()

,SMILES,target
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,1
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,1
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,1
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,1


In [13]:
to_be_printed.to_csv('data_no_agg.csv')

In [14]:
#to_be_printed.to_csv('Blood_brain_barrier_penetration.csv')

## Data aggregation

In [15]:
import pandas as pd
import numpy as np

In [16]:
data_ = pd.read_csv('data_no_agg.csv')

In [17]:
data_.head()

,Unnamed: 0,SMILES,target
0,0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,1
1,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1
2,2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,1
3,3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,1
4,4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,1


In [18]:
data_['SMILES'].describe()

count         2053
unique        2050
top       C1CCCCC1
freq             2
Name: SMILES, dtype: object

In [19]:
data_grouped = data_.groupby('SMILES')

In [20]:
## Helping functions
def get_levels(x):
    y = x.dropna()
    if y.empty:
        return np.nan
    else:
        levs = list(set(y.tolist()))
        if len(levs) == 1:
            return levs[0]
        else:
            return 'mix'

def get_all_levels(x):
    y = x.dropna()
    if y.empty:
        return np.nan
    else:
        return sorted(list(set(y.tolist())))
                
def get_mean(x):
    y = x.dropna()
    if y.empty:
        return np.nan
    else:
        return np.mean(y)
    
def expand_activity(x):
    expanded = [None for iii in range(max_level_activity)]
    n_act = len(x)
    expanded[:n_act] = x[:n_act]
    return np.array(expanded)

In [21]:
aggFunctions = {
    'target': ['min', 'max', get_all_levels, get_mean],
    'SMILES': get_levels
}
data_aggregated = data_grouped.agg(aggFunctions)
max_level_activity = max(data_aggregated[("target", "get_all_levels")].apply(lambda x: len(x)))
data_aggregated.columns = [str(item[0])+"_"+str(item[1]) for item in data_aggregated.columns.to_flat_index().to_list()]
all_activities = data_aggregated[("target_get_all_levels")].apply(lambda x: expand_activity(x))
all_activities = pd.DataFrame(all_activities.to_list(), columns=["target_"+str(iii) for iii in range(max_level_activity)])
all_activities = all_activities.set_index(data_aggregated.index)
data_aggregated = pd.concat([data_aggregated, all_activities], axis=1)
data_aggregated

,target_min,target_max,target_get_all_levels,target_get_mean,SMILES_get_levels,target_0
SMILES,,,,,,
BrCC(O)(Br)Br,1,1,[1],1,BrCC(O)(Br)Br,1
Brc1ccc2NC(=O)CN=C(c3ccccn3)c2c1,1,1,[1],1,Brc1ccc2NC(=O)CN=C(c3ccccn3)c2c1,1
C(=C)OC=C,1,1,[1],1,C(=C)OC=C,1
C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1,1,[1],1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1
C(Br)(Br)Br,1,1,[1],1,C(Br)(Br)Br,1
...,...,...,...,...,...,...
n1c(scc1)CCN,1,1,[1],1,n1c(scc1)CCN,1
n1c(scc1c1ccccc1)CCN,0,0,[0],0,n1c(scc1c1ccccc1)CCN,0
s1c(ncc1)CCN,1,1,[1],1,s1c(ncc1)CCN,1


In [22]:
data_aggregated.columns.to_list()

['target_min',
 'target_max',
 'target_get_all_levels',
 'target_get_mean',
 'SMILES_get_levels',
 'target_0']

In [23]:
to_be_saved_ = data_aggregated.columns.to_list()[5:]

In [24]:
to_save = data_aggregated[to_be_saved_]

In [25]:
to_save.reset_index()

,SMILES,target_0
0,BrCC(O)(Br)Br,1
1,Brc1ccc2NC(=O)CN=C(c3ccccn3)c2c1,1
2,C(=C)OC=C,1
3,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1
4,C(Br)(Br)Br,1
...,...,...
2045,n1c(scc1)CCN,1
2046,n1c(scc1c1ccccc1)CCN,0
2047,s1c(ncc1)CCN,1
2048,s1cc(CSCCN\C(NC)=[NH]\C#N)nc1\[NH]=C(\N)N,1


In [26]:
to_save.to_csv('data.csv')